In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks that can be used to efficiently test code for bugs and errors. It can also provide tools for generating test cases, running tests, and analyzing the results. Additionally, Langsmith can assist in setting up continuous integration pipelines for automated testing and deployment, helping to ensure that code changes do not introduce new bugs or issues. Overall, Langsmith can streamline the testing process and help developers quickly identify and fix any issues in their code.')

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    "system", "You are world class technical documentation writer.",
    "user", "{input}"
])

In [5]:
chain = prompt | llm

In [9]:
chain.invoke({"input": "how can I learn a new language"})

AIMessage(content="Learning a new language can be a rewarding and challenging experience. Here are some steps to help you get started:\n\n1. Choose a language: Decide which language you want to learn based on your interests, future goals, and practicality.\n\n2. Set goals: Determine why you want to learn the language and set specific, achievable goals for yourself.\n\n3. Find resources: There are many resources available to help you learn a new language, such as language learning apps, online courses, textbooks, language exchange programs, and language classes.\n\n4. Practice regularly: Consistent practice is key to language learning. Try to incorporate the language into your daily routine by listening to music, watching movies or TV shows, reading books or articles, and practicing speaking with native speakers.\n\n5. Immerse yourself: Immerse yourself in the language as much as possible by surrounding yourself with native speakers, visiting countries where the language is spoken, or p

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [30]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [19]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [37]:
response = retriever_chain.invoke({"input": "how can langsmith help with testing?"}) # returns a diccionary
print(response["answer"])

LangSmith can help with testing by allowing developers to create datasets, run tests on their LLM applications, upload test cases in bulk, create custom evaluations, compare different configurations, use the playground for rapid iteration and experimentation, collect feedback from users, annotate traces, add runs to datasets, and monitor key metrics over time for A/B testing.


In [20]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"), # previous chat history
    ("user", "{input}"), # currect query
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation") # this is just in orfer to create a new query but only for the retriver
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [14]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"), 
    AIMessage(content="Yes!")
    ]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me now"
})

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!")
]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [23]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search", # name of the tool
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!" # description
)

In [ ]:
# export TAVILY_API_KEY:... # it's declared in the .env file

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [24]:
tools = [retriever_tool, search]

In [25]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - 
prompt = hub.pull("hwchase17/openai-functions-agent")
llm =ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [27]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629', 'content': 'Get the latest weather conditions and outlook for San Francisco, CA, including temperature, humidity, wind, pressure, and cloud cover. See alerts, sunrise and sunset times, and historical data for the city.'}, {'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San francisco,CA', 'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 1:26 pm PDT Mar 16, 2024. Forecast Valid: 2pm PDT Mar 16, 2024-6pm PDT Mar 22, 2024 .'}, {'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco', 'content': 'Get the latest weather information for San Fr

{'input': 'what is the weather in SF?',
 'output': 'I found some sources where you can check the weather in San Francisco:\n\n1. [AccuWeather - San Francisco Current Weather](https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629): Get the latest weather conditions and outlook for San Francisco, CA, including temperature, humidity, wind, pressure, and cloud cover.\n\n2. [NOAA National Weather Service - San Francisco Forecast](https://forecast.weather.gov/zipcity.php?inputstring=San%20francisco,CA): Provides forecasts for San Francisco, CA, including local, graphical, aviation, marine, rivers and lakes, hurricanes, severe weather, and fire weather information.\n\n3. [Weather Underground - San Francisco Weather Information](https://www.wunderground.com/forecast/us/ca/san-francisco): Get the latest weather information for San Francisco, CA, including temperature, precipitation, pressure, humidity, and more. View the 10-day calendar forecast, almanac, and nearby weath

In [ ]:
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!")
    ]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})